# Generating useful Insights on Amazon Reviews

**Business Context:** This notebook show the analysis prior to model building 

## Import Libraries and Create Spark Session

In [1]:
!pip install -r requirements.txt

  Preparing metadata (setup.py) ... done
  Obtaining dependency information for azure-eventhub from https://files.pythonhosted.org/packages/66/b2/89671d93ba3c5af4a776350378765582490c3748eef3aa3a4582c3e87488/azure_eventhub-5.11.3-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 4.5 MB/s eta 0:00:00
  Obtaining dependency information for azure-core<2.0.0,>=1.14.0 from https://files.pythonhosted.org/packages/c3/0a/32b17d776a6bf5ddaa9dbad0e88de9d28a55bec1d37b8d408cc7d2e5e28d/azure_core-1.28.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.9/308.9 kB 6.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.4/185.4 kB 6.8 MB/s eta 0:00:00
  Created wheel for glob2: filename=glob2-0.7-py2.py3-none-any.whl size=9299 sha256=fb976b8fab884255708d620fe6d40a5da2f826ec7e081b69bfecb94a3cafbd3e
  Stored in directory: /home/jovyan/.cache/pip/wheels/96/41/06/9f8fddc6eb1d75bde63db7f491311a4ae26905212617e06eb2
Successfully

In [2]:
import findspark
import pyspark
findspark.init()

import pandas as pd

In [3]:
# Import pyspark.sql library
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [4]:
# Create Session
spark = SparkSession.builder.appName("AmazonReviewsDatathon").getOrCreate()

In [5]:
spark

## Loading Amazon Reviews and Metadata

In [6]:
from glob2 import glob

In [7]:
#Read all the json files from both sources of data
jsonFiles_reviews = glob('amazon_reviews/source-files/amazon_reviews/**/**/*.json.gz')
jsonFiles_meta = glob('amazon_reviews/source-files/amazon_metadata/**/*.json.gz')

In [8]:
#Check the lists of partitioned json files 
print("jsonFiles_reviews: ")
print(jsonFiles_reviews[0:5])
print("---------------")
print("jsonFiles_meta: ")
print(jsonFiles_meta[0:5])

jsonFiles_reviews: 
['amazon_reviews/source-files/amazon_reviews/partition_1/part-00000-tid-9136122565017344171-3f98196e-e0c5-4bb5-90cc-d523170ef713-86080-1-c000.json.gz', 'amazon_reviews/source-files/amazon_reviews/partition_10/part-00000-tid-698064602200227711-29b88890-b701-4ddb-82cf-535e4b44c9cf-89301-1-c000.json.gz', 'amazon_reviews/source-files/amazon_reviews/partition_10/part-00001-tid-698064602200227711-29b88890-b701-4ddb-82cf-535e4b44c9cf-89302-1-c000.json.gz', 'amazon_reviews/source-files/amazon_reviews/partition_100/part-00000-tid-6076830777214137320-043be143-2d8f-4c2a-a619-bb4ab95fa4bb-121464-1-c000.json.gz', 'amazon_reviews/source-files/amazon_reviews/partition_100/part-00011-tid-6076830777214137320-043be143-2d8f-4c2a-a619-bb4ab95fa4bb-121475-1-c000.json.gz']
---------------
jsonFiles_meta: 
['amazon_reviews/source-files/amazon_metadata/json_files/part-00000-tid-1001410877349735942-21bdbaae-698f-415f-9a21-69d6c2866481-1675-1-c000.json.gz', 'amazon_reviews/source-files/amazo

In [14]:
#Load the data from Amazon reviews using spark
df_reviews = spark.read.option("multiline","false").option("compression", "gzip").json(jsonFiles_reviews)

In [15]:
#Load the metadata using spark
df_meta = spark.read.option("multiline","false").option("compression", "gzip").json(jsonFiles_meta)

## Examining the Data 

### Amazon Reviews

In [16]:
# Print df_reviews Schema to see what attributes we have and what data type they belong
df_reviews.printSchema()

root
 |-- asin: string (nullable = true)
 |-- image: string (nullable = true)
 |-- overall: string (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- style: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: string (nullable = true)
 |-- verified: string (nullable = true)
 |-- vote: string (nullable = true)



In [49]:
# Check raw data
df_reviews.show(5)

+----------+-----+-------+--------------------+--------------+---------------+--------------------+--------------------+--------------+--------+----+
|      asin|image|overall|          reviewText|    reviewerID|   reviewerName|               style|             summary|unixReviewTime|verified|vote|
+----------+-----+-------+--------------------+--------------+---------------+--------------------+--------------------+--------------+--------+----+
|B00005QDPX| null|    5.0|Crazy Taxi is by ...|A36TDX8DY2XK5Q|       Some Kid|                null|It's Party Time! ...|    1054252800|   false|null|
|B000MXIMKK| null|    4.0|I love these pant...| AZZ1YPRM2FAUH| David Colgrove|{"Size:":" 33W x ...|Durable, good loo...|    1459900800|    true|   6|
|B00005QDPX| null|    2.0|Anyone who hasn't...|A3PASG15BRR40D|       SleepyJD|                null|A fun game that g...|    1049414400|   false|null|
|B000MXIMKK| null|    5.0|Great pants and n...|A388UHKJN07JJT|            TJK|{"Size:":" 42W x ...| 

In [18]:
# Count records number 
df_reviews.count()

13367987

We notice that `df_reviews` is a huge dataset with over 270MM records. We note that some features are categorical so we consider important to make a brief exploration to get the distribution of them (*overall* and *verified*). After that, we consider to drop some columns in order to shorten the shape of data and make our analysis more efficient. 

In [19]:
# Register df as Temp table
temp_view_name = "tempvw_reviews"
df_reviews.createOrReplaceTempView(temp_view_name)

In [50]:
df_filtrado_reviews = df_reviews.filter(df_reviews['asin'] == 'B00005QDPX')

In [48]:
df_filtrado_reviews.filter(df_filtrado_reviews['overall'] == '5.0').show()

+----------+-----+-------+--------------------+--------------+--------------------+-----+--------------------+--------------+--------+----+
|      asin|image|overall|          reviewText|    reviewerID|        reviewerName|style|             summary|unixReviewTime|verified|vote|
+----------+-----+-------+--------------------+--------------+--------------------+-----+--------------------+--------------+--------+----+
|B00005QDPX| null|    5.0|Crazy Taxi is by ...|A36TDX8DY2XK5Q|            Some Kid| null|It's Party Time! ...|    1054252800|   false|null|
|B00005QDPX| null|    5.0|I don't own this ...|A26EI47LL1R3ZZ|    Elizabeth Harbor| null|    This game ROCKS!|    1013212800|   false|   2|
|B00005QDPX| null|    5.0|The game is chall...|A2T5OC3IBDRV1H|                Nick| null|Fun but slightly ...|    1009929600|   false|null|
|B00005QDPX| null|    5.0|This is one of th...| AEIRG3VEMYO52|      Heidi Garofalo| null|Laughs and Endorp...|    1009324800|   false|   2|
|B00005QDPX| null|  

In [44]:
num_filas_filtradas = df_filtrado_reviews.count()

In [42]:
num_filas_filtradas

28

In [20]:
# Check the total rating levels ordered 
spark.sql("""
SELECT overall, COUNT(*) as count 
FROM tempvw_reviews 
GROUP BY overall 
ORDER BY overall DESC
""").show()

+-------+-------+
|overall|  count|
+-------+-------+
|    5.0|8641176|
|    4.0|2208138|
|    3.0|1020280|
|    2.0| 592040|
|    1.0| 906352|
|    0.0|      1|
+-------+-------+



In [21]:
# Check how many reviewers the dataset has
result_query_total_reviewers = spark.sql("""
SELECT COUNT(DISTINCT reviewerID) as total_reviewers
FROM tempvw_reviews 
""")

In [173]:
result_query_total_reviewers.show()

+---------------+
|total_reviewers|
+---------------+
|       34105431|
+---------------+



In [181]:
# Check how many verified and not verified reviewers the dataset has
spark.sql("""
SELECT COUNT(DISTINCT reviewerID) as total_reviewers, verified
FROM tempvw_reviews 
GROUP BY verified
""").show()

+---------------+--------+
|total_reviewers|verified|
+---------------+--------+
|       10954035|   false|
|       28705780|    true|
+---------------+--------+



In [23]:
# Check how many verified and not verified reviewers the dataset has
spark.sql("""
SELECT COUNT(DISTINCT reviewerID) as total_reviewers, verified
FROM tempvw_reviews 
GROUP BY verified
""").show()

+---------------+--------+
|total_reviewers|verified|
+---------------+--------+
|        1831052|   false|
|        6478920|    true|
+---------------+--------+



In [24]:
df_reviews.columns

['asin',
 'image',
 'overall',
 'reviewText',
 'reviewerID',
 'reviewerName',
 'style',
 'summary',
 'unixReviewTime',
 'verified',
 'vote']

In [25]:
# Get the reviewTime converting from unixtime to datetime 
new_df_reviews = df_reviews.withColumn("reviewTime", from_unixtime("unixReviewTime", "yyyy-MM-dd HH:mm:ss"))

In [26]:
new_df_reviews.show(5)

+----------+-----+-------+--------------------+--------------+---------------+--------------------+--------------------+--------------+--------+----+-------------------+
|      asin|image|overall|          reviewText|    reviewerID|   reviewerName|               style|             summary|unixReviewTime|verified|vote|         reviewTime|
+----------+-----+-------+--------------------+--------------+---------------+--------------------+--------------------+--------------+--------+----+-------------------+
|B00005QDPX| null|    5.0|Crazy Taxi is by ...|A36TDX8DY2XK5Q|       Some Kid|                null|It's Party Time! ...|    1054252800|   false|null|2003-05-30 00:00:00|
|B000MXIMKK| null|    4.0|I love these pant...| AZZ1YPRM2FAUH| David Colgrove|{"Size:":" 33W x ...|Durable, good loo...|    1459900800|    true|   6|2016-04-06 00:00:00|
|B00005QDPX| null|    2.0|Anyone who hasn't...|A3PASG15BRR40D|       SleepyJD|                null|A fun game that g...|    1049414400|   false|null|2

In [27]:
# Drop the attributes that we consider least relevant
new_df_reviews_2 = new_df_reviews.drop("image", "style", "unixReviewTime", "reviewerName")

In [28]:
new_df_reviews_2.show(5)

+----------+-------+--------------------+--------------+--------------------+--------+----+-------------------+
|      asin|overall|          reviewText|    reviewerID|             summary|verified|vote|         reviewTime|
+----------+-------+--------------------+--------------+--------------------+--------+----+-------------------+
|B00005QDPX|    5.0|Crazy Taxi is by ...|A36TDX8DY2XK5Q|It's Party Time! ...|   false|null|2003-05-30 00:00:00|
|B000MXIMKK|    4.0|I love these pant...| AZZ1YPRM2FAUH|Durable, good loo...|    true|   6|2016-04-06 00:00:00|
|B00005QDPX|    2.0|Anyone who hasn't...|A3PASG15BRR40D|A fun game that g...|   false|null|2003-04-04 00:00:00|
|B000MXIMKK|    5.0|Great pants and n...|A388UHKJN07JJT|       Great Product|    true|null|2016-04-06 00:00:00|
|B00005QDPX|    4.0|In this game you ...|A29YLR71A42A3N|Fun and Entertaining|   false|   3|2002-12-26 00:00:00|
+----------+-------+--------------------+--------------+--------------------+--------+----+-------------

In [29]:
# Register new_df as Temp table
temp_view_name_2 = "new_tempvw_reviews"
new_df_reviews_2.createOrReplaceTempView(temp_view_name_2)

In [31]:
# Check how many verified and not verified reviewers the dataset has
df_reviews_final = spark.sql("""
SELECT DISTINCT asin, overall, reviewText, reviewerID, summary, verified, vote, reviewTime
FROM new_tempvw_reviews 
""")

In [32]:
# Register df as Temp table
temp_view_name_final = "tempvw_reviews_final"
df_reviews_final.createOrReplaceTempView(temp_view_name_final)

In [33]:
df_reviews_final.count()

13336078

In [99]:
df_copy_reviews_final = df_reviews_final.alias("df_reviews_final_copy")

In [98]:
temp_view_name_final_copy = "tempvw_reviews_final_copy"
df_reviews_final.createOrReplaceTempView(temp_view_name_final_copy)

In [ ]:
# Check how many verified and not verified reviewers the dataset has
spark.sql("""
SELECT COUNT(*), verified
FROM tempvw_reviews_final_copy 
GROUP BY verified
""").show()

In [100]:
df_copy_reviews_final.show()

+----------+-------+--------------------+--------------+--------------------+--------+----+-------------------+
|      asin|overall|          reviewText|    reviewerID|             summary|verified|vote|         reviewTime|
+----------+-------+--------------------+--------------+--------------------+--------+----+-------------------+
|0312538715|    5.0|When my 73 year m...|A2B8IX5KTDI33E|Fantastic for an ...|   false|  12|2008-04-19 00:00:00|
|0330462709|    5.0|This was a really...|A1KOGB4O8NDUPY|          Five Stars|    true|null|2017-11-15 00:00:00|
|0345504976|    1.0|I have never felt...| AJFAX0DUC2MAZ| Passage To Nowhere!|    true|   8|2010-09-22 00:00:00|
|0345504976|    1.0|The first part of...|A24IWZ1KIQQ4V7|            Drags on|    true|   2|2013-04-18 00:00:00|
|0345504976|    1.0|This story starte...|A3N0GNIS8PJ51Q| Unbelievably Boring|    true|   4|2013-01-17 00:00:00|
|0345504976|    2.0|I have to admit, ...| ADGFHAEZYURLV|Cronin on To The ...|   false|null|2010-07-30 00

### Amazon Meta

In [51]:
# Print df_meta Schema to see what attributes we have and what data type they belong
df_meta.printSchema()

root
 |-- also_buy: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- also_view: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- asin: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- category: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- date: string (nullable = true)
 |-- description: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- details: struct (nullable = true)
 |    |-- \n    Item Weight: \n    : string (nullable = true)
 |    |-- \n    Package Dimensions: \n    : string (nullable = true)
 |    |-- \n    Product Dimensions: \n    : string (nullable = true)
 |    |--  Date first listed on Amazon:: string (nullable = true)
 |    |--  UNSPSC Code:: string (nullable = true)
 |    |-- ASIN:: string (nullable = true)
 |    |-- ASIN: : string (nullable = true)
 |    |-- Accessory:: string (nullable = true)
 |    |-- Age Range:: string (nullable = true)


We see `df_meta` offers some relevant description about the products and categories such that we can summarize our data in a structured way, so we need to drop the fields that are unnecessary to our analysis 

In [52]:
# Check the content within df_meta
df_meta.show(5)

+--------------------+---------+----------+-------+--------------------+----------------+--------------------+--------------------+--------------------+---+--------------------+----------+------+--------------------+--------------------+--------------------+-----+--------------------+
|            also_buy|also_view|      asin|  brand|            category|            date|         description|             details|             feature|fit|               image|  main_cat| price|                rank|        similar_item|               tech1|tech2|               title|
+--------------------+---------+----------+-------+--------------------+----------------+--------------------+--------------------+--------------------+---+--------------------+----------+------+--------------------+--------------------+--------------------+-----+--------------------+
|[B000C2AH9K, B000...|       []|B000C2E3PY|Fel-Pro|[Automotive, Repl...|November 8, 2005|[Fel-Pro gaskets ...|{null, null, null...|[PermaTorqu

In [80]:
# Create temp view to query data
temp_view_name = "tempvw_meta"
df_meta.createOrReplaceTempView(temp_view_name)

In [81]:
filtered_df_meta = spark.sql("""
SELECT DISTINCT asin, brand, main_cat, price, rank, description, category, title
FROM tempvw_meta 
WHERE main_cat NOT LIKE '%https%' AND main_cat NOT LIKE ''
""")

In [82]:
filtered_temp_view_name = "filtered_tempvw_meta"
filtered_df_meta.createOrReplaceTempView(filtered_temp_view_name)

In [87]:
resultado_consulta_meta = spark.sql("""
SELECT main_cat, COUNT(*) as conteo_registros 
FROM filtered_tempvw_meta 
GROUP BY main_cat
ORDER BY COUNT(*) DESC
""")

In [110]:
resultado_consulta_meta.show()

+----------------+
|conteo_registros|
+----------------+
|        30046118|
+----------------+



In [84]:
resultado_consulta_meta.show()

+----------------+
|conteo_registros|
+----------------+
|         3672407|
+----------------+



In [88]:
resultado_consulta_meta.show(30)

+--------------------+----------------+
|            main_cat|conteo_registros|
+--------------------+----------------+
|               Books|          865313|
|         Amazon Home|          560683|
|   Sports & Outdoors|          320070|
|        Toys & Games|          266214|
|          Automotive|          264249|
|Cell Phones & Acc...|          220627|
|        Buy a Kindle|          187580|
|             Grocery|          143295|
|Tools & Home Impr...|          115173|
|         Movies & TV|          102242|
|           Computers|           97931|
|     Office Products|           85375|
|     All Electronics|           72880|
|Arts, Crafts & Se...|           54483|
|      Camera & Photo|           45520|
| Musical Instruments|           40240|
|Industrial & Scie...|           40172|
|        Pet Supplies|           39423|
|Home Audio & Theater|           38213|
|Health & Personal...|           26557|
|         Video Games|           26262|
|                Baby|           10884|


In [89]:
filtered_df_meta.show()

+----------+-------------------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+
|      asin|              brand|            main_cat|  price|                rank|         description|            category|               title|
+----------+-------------------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+
|B000BOC9ZO|           Longacre|          Automotive|       |[">#1,044,295 in ...|[Standard 2&quot ...|[Automotive, Repl...|Longacre 50451 St...|
|B000BOYCN6|          Valvoline|          Automotive|  $4.97|[">#22,706 in Aut...|[Valvoline MaxLif...|[Automotive, Oils...|Valvoline VV324 1...|
|B000BPTV3Q|            Unknown|Tools & Home Impr...| $33.85|[">#3,231,210 in ...|[The Roadcraft<re...|[Automotive, Exte...|Unknown 60111-01 ...|
|B000BPVVAM|              Niteo|          Automotive| $10.04|[">#838,991 in Au...|[Gunk DOT 3 Super...|[Automotive, Oils...|

In [93]:
#Check if there is any duplicated asin (primary key of meta df)
conteo_duplicados = filtered_df_meta.groupBy('asin').count()
duplicated = conteo_duplicados.filter(col("count") > 1)

In [96]:
duplicated.show()

+----+-----+
|asin|count|
+----+-----+
+----+-----+



## Combine both Amazon Sources (Batch)

In [103]:
# Inner Join to combine Reviews with Meta
df_reviews_master_table = df_copy_reviews_final.join(filtered_df_meta, on=["asin"], how="inner")

In [104]:
df_reviews_master_table.printSchema()

root
 |-- asin: string (nullable = true)
 |-- overall: string (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- verified: string (nullable = true)
 |-- vote: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- main_cat: string (nullable = true)
 |-- price: string (nullable = true)
 |-- rank: string (nullable = true)
 |-- description: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- category: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- title: string (nullable = true)



In [ ]:
# Check the results 
df_reviews_master_table.show()

## Export data to Parquet 

In [92]:
path_parquet = 'parquet_meta'
partition_column = 'main_cat'
df_reviews_master_table.write.partitionBy(partition_column).option("compression", "gzip").parquet(path_parquet)